In [1]:
import torch
import numpy
import json
from tqdm import tqdm
import os
import gc
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel
import torch
from vllm import LLM, SamplingParams, LLMEngine

/home/anamikaghosh_umass_edu/.conda/envs/gpu-env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-18 13:59:19,368	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [3]:
num_gpus = torch.cuda.device_count()
print(num_gpus)

4


In [4]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("CoIR-Retrieval/cosqa-queries-corpus")
print(dataset.keys())
new_dataset=pd.DataFrame(dataset['corpus']['text'])
new_dataset_queries=pd.DataFrame(dataset['queries']['text'])
print(new_dataset_queries)

ModuleNotFoundError: No module named 'datasets'

In [ ]:
doc_code_df = pd.DataFrame({
    "doc": new_dataset_queries.iloc[:, 0],
    "code": new_dataset.iloc[:, 0]
})

doc_code_df.head(5)
doc_code_df.to_csv("cosqa-queries-corpus.csv", index=False)


In [4]:
# Set environment variable to help with memory fragmentation.
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"


def clean_output(text,keyword):
    # keyword = "Answer:"
    index = text.rfind(keyword)  # Find the last occurrence of "Answer:"
    if index != -1:
        return text[index + len(keyword):].strip()  
    return text


class ExplanationGeneratorLama:
    def __init__(self, model_name, max_new_tokens=500):
        # self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        # self.tokenizer.padding_side = "left"
        
        self.max_new_tokens = max_new_tokens
        self.sampling_params = SamplingParams(
            temperature=0.7,
            top_p=0.95,
            max_tokens=max_new_tokens
        )
        self.llm = LLM(
            model=model_name,
            dtype="half"  # using half precision for GPU
        )
        # self.engine = LLMEngine(model_name, sampling_params=self.sampling_params)

    def generate_explanations_batch(self, entries, max_new_tokens=500):
        # Create prompts by combining each entry with each prompt template.
        
        prompts = []
        
        for entry in entries:
            prompt_templates = [
                f"Doc string: {entry['doc']}\n"
                f"Code snippet: {entry['code']}\n"
                "Instruction: Provide a concise explanation of what the above doc and code mean. "
                "Generate strictly less than 100 words in total. Please give the output just as text only. Do not return anything else.\n"
                "Answer: \n"
                , 

                f"Doc string: {entry['doc']}\n"
                f"Code snippet: {entry['code']}\n"
                "Instruction: Provide a detailed line-by-line explanation of this code snippet, describing the purpose and functionality of each statement, function, and control structure. "
                "Please give the output just as text only. Do not return anything else.\n"
                "Answer: \n"
                ,

                f"Doc string: {entry['doc']}\n"
                f"Code snippet: {entry['code']}\n"
                "Instruction: Summarize what this code snippet does in simple, non-technical language, focusing on its overall purpose and key operations for someone with little programming experience. "
                "Please give the output just as text only. Do not return anything else.\n"
                "Answer: \n"
                ,

                f"Doc string: {entry['doc']}\n"
                f"Code snippet: {entry['code']}\n"
                "Instruction: Generate an explanation of the code snippet in such a way that it can regenerate the code based on this explanation. "
                "Please give the output just as text only. Do not return anything else.\n"
                "Answer: \n"
            ]
            
            for template in prompt_templates:
                prompt = (
                    f"Doc string: {entry['doc']}\n"
                    f"Code snippet: {entry['code']}\n"
                    f"{template}\n"
                    "Answer: \n"
                )
                prompts.append(prompt)
                
        # results = self.engine.generate(prompts)
        results = self.llm.generate(prompts, self.sampling_params)
        explanations = [result.outputs[0].text for result in results]
        
        # Regroup explanations by entry.
        n_prompts = len(prompt_templates)
        grouped_explanations = []
        for i in range(0, len(explanations), n_prompts):
            grouped_explanations.append(explanations[i:i+n_prompts])
            
        return grouped_explanations


In [5]:
import pandas as pd
import torch, gc
from tqdm import tqdm
# Import or define your ExplanationGeneratorLama class here

if __name__ == "__main__":
    csv_path = "/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/cosqa_queries_code_corpus.csv"  # change this to your CSV input path
    output_csv_path = "/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/CoSQA_granite_explanations_vllm.csv"
    
    # Load the CSV file into a DataFrame
    df = pd.read_csv(csv_path).head(20)
    print("Data loaded from CSV")
    

    models_dict = {
        # "deepseek": "/datasets/ai/deepseek/hub/models--deepseek-ai--DeepSeek-R1-Distill-Qwen-1.5B/snapshots/530ca3e1ad39d440e182c2e4317aa40f012512fa",
        'granite': "/datasets/ai/ibm-granite/hub/models--ibm-granite--granite-3.0-2b-instruct/snapshots/69e41fe735f54cec1792de2ac4f124b6cc84638f"

    }    
    
    batch_size = 4
    
    for model_key, model_path in tqdm(models_dict.items(), desc="Processing models"):
        print(f"\nProcessing model {model_key}...")
        generator = ExplanationGeneratorLama(model_path)
        if hasattr(generator, 'model'):
            generator.model.eval()
        
        # Process the DataFrame in batches
        for i in tqdm(range(0, len(df), batch_size), desc="Processing batches"):
            # Create a batch of entries (each is a dict with "doc" and "code")
            batch_entries = df.iloc[i:i+batch_size][["corpus_id", "query_id", "doc", "code"]].to_dict("records")
            
            # Wrap inference in a no_grad context to prevent gradient computations.
            with torch.no_grad():
                batch_explanations = generator.generate_explanations_batch(batch_entries)
            
            for j, explanation_variants in enumerate(batch_explanations):
                for idx, raw_text in enumerate(explanation_variants):
                    # print(raw_text)
                    cleaned_text = clean_output(raw_text,"Answer:")
                    cleaned_text = clean_output(cleaned_text,"</think>")
                    df.loc[i+j, f'explanation_{model_key}_{idx+1}'] = cleaned_text
            
            torch.cuda.empty_cache()
            gc.collect()
        
        del generator
        torch.cuda.empty_cache()
        gc.collect()
    
    # Save the DataFrame with the new explanation columns to a CSV file
    # df.to_csv(output_csv_path, index=False)
    # print(f"\nExplanations from all models have been saved to {output_csv_path}")


Data loaded from CSV


Processing models:   0%|          | 0/1 [00:00<?, ?it/s]


Processing model granite...
INFO 03-18 13:59:23 __init__.py:207] Automatically detected platform cuda.
WARNING 03-18 13:59:23 config.py:2448] Casting torch.bfloat16 to torch.float16.
INFO 03-18 13:59:45 config.py:549] This model supports multiple tasks: {'reward', 'classify', 'embed', 'score', 'generate'}. Defaulting to 'generate'.
INFO 03-18 13:59:45 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.3) with config: model='/datasets/ai/ibm-granite/hub/models--ibm-granite--granite-3.0-2b-instruct/snapshots/69e41fe735f54cec1792de2ac4f124b6cc84638f', speculative_config=None, tokenizer='/datasets/ai/ibm-granite/hub/models--ibm-granite--granite-3.0-2b-instruct/snapshots/69e41fe735f54cec1792de2ac4f124b6cc84638f', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_cu

[W318 13:59:48.843524685 CUDAAllocatorConfig.h:28] Warning: expandable_segments not supported on this platform (function operator())

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:14<00:14, 14.05s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:14<00:00,  7.27s/it]



INFO 03-18 14:00:04 model_runner.py:1115] Loading model weights took 4.7199 GB
INFO 03-18 14:00:06 worker.py:267] Memory profiling takes 1.71 seconds
INFO 03-18 14:00:06 worker.py:267] the current vLLM instance can use total_gpu_memory (10.75GiB) x gpu_memory_utilization (0.90) = 9.67GiB
INFO 03-18 14:00:06 worker.py:267] model weights take 4.72GiB; non_torch_memory takes 0.05GiB; PyTorch activation peak memory takes 0.47GiB; the rest of the memory reserved for KV Cache is 4.43GiB.
INFO 03-18 14:00:06 executor_base.py:111] # cuda blocks: 3627, # CPU blocks: 3276
INFO 03-18 14:00:06 executor_base.py:116] Maximum concurrency for 4096 tokens per request: 14.17x
INFO 03-18 14:00:10 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_utili

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:21<00:00,  1.65it/s]

INFO 03-18 14:00:31 model_runner.py:1562] Graph capturing finished in 21 secs, took 0.28 GiB
INFO 03-18 14:00:31 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 27.82 seconds















Processed prompts: 100%|██████████| 16/16 [00:07<00:00,  2.14it/s, est. speed input: 479.03 toks/s, output: 440.98 toks/s]












Processed prompts: 100%|██████████| 16/16 [00:07<00:00,  2.02it/s, est. speed input: 603.91 toks/s, output: 574.04 toks/s]













Processed prompts: 100%|██████████| 16/16 [00:07<00:00,  2.15it/s, est. speed input: 408.36 toks/s, output: 518.09 toks/s]














Processed prompts: 100%|██████████| 16/16 [00:07<00:00,  2.27it/s, est. speed input: 529.98 toks/s, output: 522.17 toks/s]













Processing models: 100%|██████████| 1/1 [01:48<00:00, 108.04s/it]


In [6]:
df.head()

,query_id,doc,corpus_id,code,explanation_granite_1,explanation_granite_2,explanation_granite_3,explanation_granite_4
0,q1,python code to write bool value 1,d1,"def writeBoolean(self, n):\n """"""\n ...",\nThe code snippet is a method named `writeBoo...,"\n```python\ndef writeBoolean(self, n):\n ""...",\nThis code snippet is a part of a class that ...,\nThis Python code snippet defines a method ca...
1,q2,"""python how to manipulate clipboard""",d2,"def paste(xsel=False):\n """"""Returns system ...",\nThe provided Python code snippet defines a f...,\n1. `def paste(xsel=False):` - This line defi...,\nThis code snippet is a function in Python th...,\nThis Python code snippet defines a function ...
2,q3,python colored output to html,d3,"def _format_json(data, theme):\n """"""Pretty ...",\nThe provided Python code defines a function ...,"\n1. `def _format_json(data, theme):` - This l...","\nThis code snippet, named `_format_json`, is ...",\nThis code snippet defines a function called ...
3,q4,"python ""create directory"" using ""relative path""",d4,"def create_path(path):\n """"""Creates a absol...",\nThe provided code defines a function called ...,\nThis code snippet defines a function called ...,\nThis code snippet helps create a new directo...,\nThe provided code snippet defines a function...
4,q5,python column of an array,d5,"def _vector_or_scalar(x, type='row'):\n """"""...",\nThe provided Python code defines a function ...,"\n1. `def _vector_or_scalar(x, type='row')`: T...",\nThis code snippet helps convert a list or a ...,\nThe code snippet provided is a Python functi...


In [1]:
from vllm import LLM, SamplingParams

llm = LLM(
    model="/datasets/ai/deepseek/hub/models--deepseek-ai--DeepSeek-R1-Distill-Qwen-1.5B/snapshots/530ca3e1ad39d440e182c2e4317aa40f012512fa",
    dtype="half"  # using half precision for GPU
)

sampling_params = SamplingParams(temperature=0.7, top_p=0.9, max_tokens=1000)

outputs = llm.generate(["What is AI?"], sampling_params)

print(outputs)


/home/pmangina_umass_edu/.conda/envs/gpu-env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-17 14:30:44,152	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 03-17 14:30:45 __init__.py:207] Automatically detected platform cuda.
WARNING 03-17 14:30:45 config.py:2448] Casting torch.bfloat16 to torch.float16.
INFO 03-17 14:31:00 config.py:549] This model supports multiple tasks: {'generate', 'classify', 'reward', 'score', 'embed'}. Defaulting to 'generate'.
WARNING 03-17 14:31:00 arg_utils.py:1187] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 03-17 14:31:00 config.py:1555] Chunked prefill is enabled with max_num_batched_tokens=2048.
INFO 03-17 14:31:00 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.3) with config: model='/datasets/ai/deepseek/hub/models--deepseek-ai--DeepSeek-R1-Distill-Qwen-1.5B/snapshots/530ca3e1ad39d440e182c2e4317aa40f012512fa', speculative_config=None, tokenizer='/datasets/ai/deepseek/hub/models--deepseek

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:09<00:00,  9.10s/it]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:09<00:00,  9.10s/it]



INFO 03-17 14:31:13 model_runner.py:1115] Loading model weights took 3.3460 GB
INFO 03-17 14:31:14 worker.py:267] Memory profiling takes 1.27 seconds
INFO 03-17 14:31:14 worker.py:267] the current vLLM instance can use total_gpu_memory (47.45GiB) x gpu_memory_utilization (0.90) = 42.70GiB
INFO 03-17 14:31:14 worker.py:267] model weights take 3.35GiB; non_torch_memory takes 0.06GiB; PyTorch activation peak memory takes 1.39GiB; the rest of the memory reserved for KV Cache is 37.91GiB.
INFO 03-17 14:31:14 executor_base.py:111] # cuda blocks: 88728, # CPU blocks: 9362
INFO 03-17 14:31:14 executor_base.py:116] Maximum concurrency for 131072 tokens per request: 10.83x
INFO 03-17 14:31:21 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:19<00:00,  1.79it/s]

INFO 03-17 14:31:40 model_runner.py:1562] Graph capturing finished in 20 secs, took 0.20 GiB
INFO 03-17 14:31:40 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 27.69 seconds



Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible


[RequestOutput(request_id=0, prompt='What is AI?', prompt_token_ids=[151646, 3838, 374, 15235, 30], encoder_prompt=None, encoder_prompt_token_ids=None, prompt_logprobs=None, outputs=[CompletionOutput(index=0, text=' Can you explain it in simple terms?\n\nWhat is the difference between AI and machine learning?\n\nWhat is the difference between AI and deep learning?\n\nWhat is the difference between AI and NLP?\n\nWhat is the difference between AI and NLP? Wait, that\'s the same as the previous question.\n\nWhat is the difference between AI and NLP?\n\nWait, that\'s the same as the previous question.\n\nWait, I think I need to clarify this.\n\nWait, no, the user is asking for a list of differences between AI and NLP, but in the initial question, they are asking for the same thing twice.\n\nSo perhaps the user is confused, and wants to know the differences between AI and NLP, and between AI and machine learning, and between AI and deep learning.\n\nSo perhaps I should explain the differen

In [4]:
!export TORCH_USE_CUDA_DSA=1  # Optional: for better error messages during debugging


In [5]:
!export CUDA_LAUNCH_BLOCKING=1
